<a href="https://colab.research.google.com/github/KimRass/Programming/blob/master/Data%20Science/Machine%20Learning/NLP/NSMC%20%26%20KoBERT%20(Sentiment%20Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# Data Science/Machine Learning/NLP/NSMC & KoBERT (Sentiment Analysis).ipynb
!pip install transformers==4.4.2
!pip install tensorflow_addons
!pip install sentencepiece

import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py", filename="tokenization_kobert.py")

('tokenization_kobert.py', <http.client.HTTPMessage at 0x7faa7a5d7d50>)

In [51]:
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir("/content")

import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, LayerNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv1DTranspose, Conv2D, Conv2DTranspose, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D, RepeatVector
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, Adam
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
# MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras import losses
# MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from transformers import TFBertModel
from tokenization_kobert import KoBertTokenizer
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import sklearn
# import logging

# logging.basicConfig(level=logging.ERROR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
os.chdir("/content/drive/MyDrive/NLP")
ratings_tr = pd.read_table("./Datasets/Naver Sentiment Movie Corpus/ratings_train.txt")
ratings_te = pd.read_table("./Datasets/Naver Sentiment Movie Corpus/ratings_test.txt")

ratings_tr = ratings_tr.dropna(subset=["document"])
ratings_te = ratings_te.dropna(subset=["document"])

In [53]:
ratings_tr

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [54]:
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

In [55]:
tokenizer.encode("평점이 낮다")

[2, 4841, 7224, 1429, 5782, 3]

In [56]:
print(tokenizer.encode("평점이 낮다", max_length=seq_len, truncation=True, padding="max_length"))

[2, 4841, 7224, 1429, 5782, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [57]:
a = tokenizer.encode("평점이 낮다", max_length=seq_len, truncation=True, padding="max_length")

In [58]:
a.index(1)

6

In [72]:
seq_len = 64
def create_bert_embeddings(df):
    df["document"] = df["document"].astype("str")
    df["label"] = df["label"].astype("int")
    
    token_embs = list()
    segment_embs = list()
    pos_embs = list()
    labels = list()
    for _, row in tqdm(df.iterrows()):
        ids = tokenizer.encode(row["document"], max_length=seq_len, truncation=True, padding="max_length")
        n_pads = ids.count(1)
        segment = [0]*seq_len
        pos = [1]*(seq_len - n_pads) + [0]*n_pads

        token_embs.append(ids)
        segment_embs.append(segment)
        pos_embs.append(pos)
        labels.append(row["label"])
    token_embs = np.array(token_embs)
    segment_embs = np.array(segment_embs)
    pos_embs = np.array(pos_embs)
    labels = np.array(labels)
    return [token_embs, pos_embs, segment_embs], labels

In [73]:
tr_X, tr_y = create_bert_embeddings(ratings_tr)
te_X, te_y = create_bert_embeddings(ratings_te)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [36]:
def create_model():
    model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

    inputs_token = Input(shape=(seq_len,), dtype=tf.int32, name="Input_token")
    inputs_pos = Input(shape=(seq_len,), dtype=tf.int32, name="Input_position")
    inputs_segment = Input(shape=(seq_len,), dtype=tf.int32, name="Input_segment")
    inputs = [inputs_token, inputs_pos, inputs_segment]

    outputs = model(inputs)
    # output[0]은 모든 step의 hidden state. => 다대다 문제
    # output[1]은 첫번째 위치의 hidden state. => 다대일 문제
    z = Dropout(rate=0.5, name="Dropout")(outputs[1])
    
    outputs = Dense(units=1, activation="sigmoid", 
                kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02),
                name="Dense")(z)
    bert_model = Model(inputs=inputs, outputs=outputs)
    bert_model.summary()

    # `total_steps`: batch_size*n_epochs
    optimizer = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps=2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)
    bert_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["acc"])
    return bert_model

In [37]:
model_path = "nsmc_kobert.h5"
if os.path.exists(model_path):
    bert_model = create_model()
    bert_model.load_weights(model_path)
else:
    # resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="grpc://" + os.environ["COLAB_TPU_ADDR"])
    # tf.config.experimental_connect_to_cluster(resolver)
    # tf.tpu.experimental.initialize_tpu_system(resolver)
    # strategy = tf.distribute.experimental.TPUStrategy(resolver)
    # with strategy.scope():
        # bert_model = create_model()

        # hist = bert_model.fit(x=tr_X, y=tr_y, validation_split=0.2, batch_size=64, epochs=3, verbose=1, shuffle=True)

        # bert_model.save_weights(model_path, overwrite=True)
    
    bert_model = create_model()

    hist = bert_model.fit(x=tr_X, y=tr_y, validation_split=0.2, batch_size=64, epochs=3, verbose=1, shuffle=True)

    bert_model.save_weights(model_path, overwrite=True)

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/optimizers/rectified_adam.py:120: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/3
 205/1875 [==>...........................] - ETA: 54:48 - loss: 0.6973 - binary_accuracy: 0.5170

KeyboardInterrupt: ignored

In [ ]:
def predict_create_bert_embeddings(data_df):
    tokens, masks, segments = [], [], []
    for _, row in data_df.iterrows():
        # token : 문장을 토큰화함
        token = tokenizer.encode(row["document"], max_length=seq_len,
                                 padding="max_length")
        n_zeros = token.count(1)
        mask = [1]*(seq_len - n_zeros) + [0]*n_zeros
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*seq_len

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)  
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    return [tokens, masks, segments]

# 위에 정의한 create_bert_embeddings 함수를 불러오는 함수를 정의
def predict_load_data(df):
    df["document"] = df["document"].astype("str")
    X = predict_create_bert_embeddings(df)
    return X

In [ ]:
preds = bert_model.predict(te_X)

In [ ]:
test_set = predict_load_data(ratings_te)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [ ]:
y_true = test["label"]
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

NameError: ignored

In [ ]:
def sentence_create_bert_embeddings(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=seq_len, pad_to_max_length=True)
    
    n_zeros = token.count(0) 
    mask = [1]*(seq_len-n_zeros) + [0]*n_zeros 
    segment = [0]*seq_len

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def movie_evaluation_predict(sentence):
    data_x = sentence_create_bert_embeddings(sentence)
    predict = bert_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      print("(부정 확률 : %.2f) 부정적인 영화 평가입니다." % (1-predict_value))
    elif predict_answer == 1:
      print("(긍정 확률 : %.2f) 긍정적인 영화 평가입니다." % predict_value)

In [ ]:
movie_evaluation_predict("즐겁습니다")

/content/notebooks/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(부정 확률 : 0.64) 부정적인 영화 평가입니다.
